# Intro to Decks: OT2
### In this tutorial, you will see how to instantiate an OT2 Deck, how to add labware, and how to move liquids around.

First we will import `LiquidHandler`, a backend called `ChatterBoxBackend` that prints the text
output of our commands, a class `Visualizer` that provides a visualization of the robot deck as we
run commands, and a class `OTDeck` that will represent the deck of an OpenTrons OT2, one of
the most widely used liquid handling robots. 

Make sure to also `import opentrons` !

### Imports

In [1]:
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import ChatterBoxBackend
from pylabrobot.visualizer.visualizer import Visualizer
from pylabrobot.resources.opentrons import OTDeck

from pylabrobot.resources.opentrons.load import *
from pylabrobot.resources.opentrons.plates import *

import opentrons

### Setting up the Deck and Visualizer

First, we will create an instance of the `LiquidHandler` class. This may take some time to set up, so we run the `setup()` function with the `await` keyword.

In [2]:
lh = LiquidHandler(backend=ChatterBoxBackend(), deck=OTDeck())

await lh.setup()

Setting up the robot.
Resource deck was assigned to the robot.
Resource trash_container was assigned to the robot.


After initializing our `LiquidHandler`, we want to create an instance of a `Visualizer`. This will allow you to see the Deck and follow your protocol in real time. You can see how tips and liquids move as you run commands. Make sure to open the `Visualizer` in another window.

In [3]:
vis = Visualizer(resource=lh)
await vis.setup()

Websocket server started at http://127.0.0.1:2122
File server started at http://127.0.0.1:1338 . Open this URL in your browser.


### Adding Labware to the Deck

Now, we are ready to add some labware to the deck. PyLabRobot has many different labware items already defined. Only import the ones that you need for your protocol. A full list of labware can be found in `PyLabRobot\Resources\opentrons`. You can also create custom labware, but that is out of scope for this tutorial.

Let's begin by importing a `TubeRack`, a `TipRack`, and a `Plate`.
<details>
    <summary>Definitions of Labware:</summary>
    
* **TubeRack** = Used to hold various tubes, commonly the 2mL Eppendorfs.

* **TipRack** = Labware that holds pipette tips.

* **Plate** = Well-Plate that one can add liquids to.
</details>

In [4]:
from pylabrobot.resources import (
    opentrons_24_tuberack_eppendorf_2ml_safelock_snapcap,
    opentrons_96_tiprack_300ul,
    corning_96_wellplate_360ul_flat
)

**Note:** The OT2 has **11 spots** for labware. It also has a built in trash can for discarding tips.

One thing to keep in mind when designing a protocol is `layout efficiency`. The more separated labware is on the deck, the longer your protocol will be because the pipette has to travel farther.

In general, you want to keep your tip racks in the back, your stocks in the middle, and then finally your plates in the front. This allows for `linear movement`.

For example, the arm grabs a **tip from slot 7**, aspirates **stock from slot 4**, and then finally **dispenses in slot 1**. Reducing the travel time of the pipette will decrease the runtime of your protocol.

To place labware on the Deck, call `assign_child_at_slot()`. Pass in the labware you want to place along with the slot you want to place it in.

In [5]:
# When you instantiate a labware, give it a name that will show when you
# mouse over it in the visualizer.
tip_rack = opentrons_96_tiprack_300ul("tip_rack")

lh.deck.assign_child_at_slot(tip_rack, 7)

Resource tip_rack was assigned to the robot.


In [6]:
# Stock Solution Tube Rack
tube_rack = opentrons_24_tuberack_eppendorf_2ml_safelock_snapcap("tube_rack")

lh.deck.assign_child_at_slot(tube_rack, 4)

Resource tube_rack was assigned to the robot.


In [7]:
# Plate for Sample Preparation
plate = corning_96_wellplate_360ul_flat("prep_plate")

lh.deck.assign_child_at_slot(plate, 1)

Resource prep_plate was assigned to the robot.


### Adding Liquids to the Deck

Let's add some liquids to our `tube_rack`. We can add up to the `max_volume` of the tube. If you go over this number, PyLabRobot will throw an error. We shall add 1000µL of 4 different dyes to the first column of our `tube_rack`. This corresponds to wells *A1, B1, C1, and D1*.

To iterate over locations on labware, we use the `traverse()` function. This produces a generator object that we use the `next` keyword on to yield our desired wells.

`traverse()` takes in two arguments: **batch_size** is the amount of wells to return, and **direction** is how to iterate over the wells. In our case, we use `"down"` to return the wells column wise.

<details>
    <summary>More info on <b>direction:</summary>

* `"down"`, `"snake_down"`, `"right"`, and `"snake_right"` start at the top left item **(A1)**.
        
* `"up"` and `"snake_up"` start at the bottom left **(H1)**.
    
* `"left"` and `"snake_left"` start at the top right **(A12)**.

* The `snake` directions alternate between going in the given direction and going in the opposite direction. For example, `"snake_down"` will go from A1 to H1, then H2 to A2, then A3 to H3, etc.
</details>

In [8]:
first_col_tubes = next(tube_rack.traverse(batch_size=4, direction='down'))
first_col_tubes

[Tube(name=tube_rack_A1, location=(018.210, 075.430, 041.270), size_x=6.223, size_y=6.223, size_z=39.1, category=tube),
 Tube(name=tube_rack_B1, location=(018.210, 056.150, 041.270), size_x=6.223, size_y=6.223, size_z=39.1, category=tube),
 Tube(name=tube_rack_C1, location=(018.210, 036.870, 041.270), size_x=6.223, size_y=6.223, size_z=39.1, category=tube),
 Tube(name=tube_rack_D1, location=(018.210, 017.590, 041.270), size_x=6.223, size_y=6.223, size_z=39.1, category=tube)]

Use the `tracker.add_liquid()` function to put liquid in a `tube`. The `tracker` class contains all of the methods associated with keeping record of how much/what kind of liquid is in a given container.

Pass in a string for **"Liquid_Type"** and a number for **Volume** to this function.

Let's add our dyes to the tubes in `first_col_tubes`.

In [9]:
for i in range(len(first_col_tubes)):
    
    first_col_tubes[i].tracker.add_liquid(f"Dye_{i}", 1000)
    
    #Commit the change
    first_col_tubes[i].tracker.commit()

Here's a Utility function for printing all of the filled spots of a `TubeRack`.

In [10]:
def print_filled_spots_of_tubeRack(tube_rack):

    all_tubes = tube_rack.get_all_children()

    all_empty = True
    
    for tube in all_tubes:
    
        liquid = tube.tracker.liquids

        if liquid != []:
            print(f"Spot {tube.name.split('_')[-1]} contains:")

            name = liquid[0][0]
            vol = liquid[0][1]
        
            print(f"{vol}uL of {name}")

            all_empty = False

    if all_empty:
        print("Entire rack is empty!")

In [11]:
print_filled_spots_of_tubeRack(tube_rack)

Spot A1 contains:
1000uL of Dye_0
Spot B1 contains:
1000uL of Dye_1
Spot C1 contains:
1000uL of Dye_2
Spot D1 contains:
1000uL of Dye_3


### Moving Liquids from Point A to Point B

Why is it not updating the contents of the tubes correctly?? The animations are also broken now.

In [12]:
import time

In [13]:
await lh.return_tips()

RuntimeError: No tips have been picked up.

In [37]:
await lh.pick_up_tips(tip_rack["A1"])
time.sleep(2)

await lh.aspirate(tube_rack["A1"], vols=[100])
tube_rack["A1"][0].tracker.commit()
time.sleep(2)

await lh.dispense(plate["A1"], vols=[100])
plate["A1"][0].tracker.commit()
time.sleep(2)

await lh.return_tips()

Picking up tips [Pickup(resource=TipSpot(name=tip_rack_A1, location=(014.380, 074.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47))].
Aspirating [Aspiration(resource=Tube(name=tube_rack_A1, location=(018.210, 075.430, 041.270), size_x=6.223, size_y=6.223, size_z=39.1, category=tube), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47), volume=100, flow_rate=None, liquid_height=None, blow_out_air_volume=None, liquids=[(None, 100)])].
Dispensing [Dispense(resource=Well(name=prep_plate_A1, location=(014.380, 074.240, 003.550), size_x=4.851, size_y=4.851, size_z=10.67, category=well), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47), volume=100, flow_rate=None, liquid_height=None, blow_out_air_volume=None, liquids=[(None, 100)])].
Dropping tips [Drop(resource

In [34]:
tube_rack["A1"]

[Tube(name=tube_rack_A1, location=(018.210, 075.430, 041.270), size_x=6.223, size_y=6.223, size_z=39.1, category=tube)]

In [38]:
print_filled_spots_of_tubeRack(tube_rack)

Spot A1 contains:
1000uL of Dye_0
Spot B1 contains:
1000uL of Dye_1
Spot C1 contains:
1000uL of Dye_2
Spot D1 contains:
1000uL of Dye_3


In [32]:
tube_rack[0][0].tracker.liquids

[('Dye_0', 1000)]

In [ ]:
await lh.pick_up_tips(tip_rack["A1"])
time.sleep(1)

await lh.aspirate(plate["A1"], vols=[100])
time.sleep(1)

await lh.dispense(tube_rack["A1"], vols=[100])
time.sleep(1)

await lh.return_tips()

### For future reference, how to export states of labware as .json files

In [16]:
""" Save the state of this resource and all children to a JSON file.

    Args:
      fn: File name. Caution: file will be overwritten.
      indent: Same as `json.dump`'s `indent` argument (for json pretty printing).

    Examples:
      Saving to a json file:

      >>> deck.save_state_to_file("my_state.json")
"""


' Save the state of this resource and all children to a JSON file.\n\n    Args:\n      fn: File name. Caution: file will be overwritten.\n      indent: Same as `json.dump`\'s `indent` argument (for json pretty printing).\n\n    Examples:\n      Saving to a json file:\n\n      >>> deck.save_state_to_file("my_state.json")\n'